#### 1.0_DATA PREPROCESSING for ChatBox

We need two list: questions and answers

In [2]:
#!/usr/bin/env python2
#building a chat box with deepNLP
import numpy as np
import tensorflow as tf
import re
import time

lines = open('movie_lines.txt',encoding = 'utf-8',errors='ignore').read().split('\n')
conversations = open('movie_conversations.txt',encoding = 'utf-8',errors='ignore').read().split('\n')

In [3]:
id2line = {}
for line in lines:
    #contemperoral variables
    _line=line.split(' +++$+++ ')
    if len(_line)==5:
        id2line[_line[0]] = _line[4]

In [17]:
#id2line#key:value=key,value of dictionary

In [21]:
conversations_ids = []
for conversation in conversations[:-1]:
    _conversation = conversation.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    conversations_ids.append(_conversation.split(','))

In [22]:
conversations_ids

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208'],
 ['L271', 'L272', 'L273', 'L274', 'L275'],
 ['L276', 'L277'],
 ['L280', 'L281'],
 ['L363', 'L364'],
 ['L365', 'L366'],
 ['L367', 'L368'],
 ['L401', 'L402', 'L403'],
 ['L404', 'L405', 'L406', 'L407'],
 ['L575', 'L576'],
 ['L577', 'L578'],
 ['L662', 'L663'],
 ['L693', 'L694', 'L695'],
 ['L696', 'L697', 'L698', 'L699'],
 ['L860', 'L861'],
 ['L862', 'L863', 'L864', 'L865'],
 ['L866', 'L867', 'L868', 'L869'],
 ['L870', 'L871', 'L872'],
 ['L924', 'L925'],
 ['L984', 'L985'],
 ['L1044', 'L1045'],
 ['L49', 'L50', 'L51'],
 ['L571', 'L572', 'L573'],
 ['L579', 'L580'],
 ['L595', 'L596', 'L597'],
 ['L598', 'L599', 'L600'],
 ['L659', 'L660'],
 ['L952', 'L953'],
 ['L394', 'L395'],
 ['L396', 'L397'],
 ['L589', 'L590', 'L591'],
 ['L592', 'L593'],
 ['L756', 'L757', 'L758'],
 ['L759', 'L760'],
 ['L164', 'L165'],
 ['L319', 'L320'],
 ['L441', 'L442', 'L443', 'L444', 'L445']

In [15]:
#before we train our model, we need to specify two list:
#input list and output list
#sequence to sequence

In [23]:
questions =  []
answers = []
for conversation in conversations_ids:
    for i in range(len(conversation)-1):
        questions.append(id2line[conversation[i]])
        answers.append(id2line[conversation[i+1]])

In [26]:
#questions
#answers

#### TEXT CLEANING

replace by full expressions

In [63]:
#cleaning steps essential
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i m","i am", text)
    text = re.sub(r"he's","he is", text)
    text = re.sub(r"she's","she is", text)
    text = re.sub(r"that's","that is", text)
    text = re.sub(r"what's","what is", text)
    text = re.sub(r"where's","where is", text)
    text = re.sub(r"\'ll"," will", text)
    text = re.sub(r"\'ve"," have", text)
    text = re.sub(r"\'d"," would", text)
    text = re.sub(r"won't"," will not", text)
    text = re.sub(r"can't"," cannot", text)
    text = re.sub(r"[-()\"#;:<>{}+=_~|!?.],","", text)
    #questions remains
    return text

In [64]:
clean_questions=[]
for question in questions :
    clean_questions.append(clean_text(question))

In [65]:
#clean_questions

In [66]:
clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer))

In [67]:
#clean_answers

remove not frequent words

In [68]:
#same 
word2count = {}
for question in clean_questions:
    for word in question.split():
        if word not in word2count:
            word2count[word]=1
        else:
            word2count[word]+=1

for answer in clean_answers:
    for word in answer.split():
        if word not in word2count:
            word2count[word]=1
        else:
            word2count[word]+=1

In [82]:
#word2count

In [93]:
#comparing with threshold(10-20)
threshold = 20

In [94]:
questionword2int = {}
word_number = 0
for word , count in word2count.items():
    if count>=threshold:
        questionword2int[word] = word_number
        word_number += 1

In [95]:
answerword2int = {}
word_number = 0
for word , count in word2count.items():
    if count>=threshold:
        answerword2int[word] = word_number
        word_number += 1

In [96]:
answerword2int

{'can': 0,
 'we': 1,
 'make': 2,
 'this': 3,
 'and': 4,
 'andrew': 5,
 'are': 6,
 'having': 7,
 'an': 8,
 'incredibly': 9,
 'public': 10,
 'up': 11,
 'on': 12,
 'the': 13,
 'again.': 14,
 'well,': 15,
 'i': 16,
 'thought': 17,
 'would': 18,
 'start': 19,
 'with': 20,
 'if': 21,
 'that': 22,
 'is': 23,
 'okay': 24,
 'you.': 25,
 'not': 26,
 'part.': 27,
 'please.': 28,
 "you're": 29,
 'asking': 30,
 'me': 31,
 'out.': 32,
 'so': 33,
 'cute.': 34,
 'what': 35,
 'your': 36,
 'name': 37,
 'again?': 38,
 'no,': 39,
 "it's": 40,
 'my': 41,
 'fault': 42,
 '--': 43,
 "didn't": 44,
 'have': 45,
 'a': 46,
 'proper': 47,
 '---': 48,
 'thing': 49,
 'is,': 50,
 "i'm": 51,
 'at': 52,
 'mercy': 53,
 'of': 54,
 'particularly': 55,
 'breed': 56,
 'loser.': 57,
 'sister.': 58,
 'cannot': 59,
 'date': 60,
 'until': 61,
 'she': 62,
 'does.': 63,
 'why?': 64,
 'mystery.': 65,
 'used': 66,
 'to': 67,
 'be': 68,
 'really': 69,
 'popular': 70,
 'when': 71,
 'started': 72,
 'high': 73,
 'school,': 74,
 'then':

In [97]:
#add tokens to two strings
#add tokens

In [98]:
tokens = ['<PAD>','<EOS>','<OUT>','<SOS>']
for token in tokens:
    questionword2int[token] = len(questionword2int) + 1
for token in tokens:
    answerword2int[token] = len(answerword2int) + 1

In [99]:
questionword2int

{'can': 0,
 'we': 1,
 'make': 2,
 'this': 3,
 'and': 4,
 'andrew': 5,
 'are': 6,
 'having': 7,
 'an': 8,
 'incredibly': 9,
 'public': 10,
 'up': 11,
 'on': 12,
 'the': 13,
 'again.': 14,
 'well,': 15,
 'i': 16,
 'thought': 17,
 'would': 18,
 'start': 19,
 'with': 20,
 'if': 21,
 'that': 22,
 'is': 23,
 'okay': 24,
 'you.': 25,
 'not': 26,
 'part.': 27,
 'please.': 28,
 "you're": 29,
 'asking': 30,
 'me': 31,
 'out.': 32,
 'so': 33,
 'cute.': 34,
 'what': 35,
 'your': 36,
 'name': 37,
 'again?': 38,
 'no,': 39,
 "it's": 40,
 'my': 41,
 'fault': 42,
 '--': 43,
 "didn't": 44,
 'have': 45,
 'a': 46,
 'proper': 47,
 '---': 48,
 'thing': 49,
 'is,': 50,
 "i'm": 51,
 'at': 52,
 'mercy': 53,
 'of': 54,
 'particularly': 55,
 'breed': 56,
 'loser.': 57,
 'sister.': 58,
 'cannot': 59,
 'date': 60,
 'until': 61,
 'she': 62,
 'does.': 63,
 'why?': 64,
 'mystery.': 65,
 'used': 66,
 'to': 67,
 'be': 68,
 'really': 69,
 'popular': 70,
 'when': 71,
 'started': 72,
 'high': 73,
 'school,': 74,
 'then':

In [102]:
#answerword2int

In [101]:
#inverse a dictionary

In [105]:
answerints2word = {w_i : w for w , w_i in answerword2int.items()}

In [107]:
answerints2word #unique intergers and unique words

{0: 'can',
 1: 'we',
 2: 'make',
 3: 'this',
 4: 'and',
 5: 'andrew',
 6: 'are',
 7: 'having',
 8: 'an',
 9: 'incredibly',
 10: 'public',
 11: 'up',
 12: 'on',
 13: 'the',
 14: 'again.',
 15: 'well,',
 16: 'i',
 17: 'thought',
 18: 'would',
 19: 'start',
 20: 'with',
 21: 'if',
 22: 'that',
 23: 'is',
 24: 'okay',
 25: 'you.',
 26: 'not',
 27: 'part.',
 28: 'please.',
 29: "you're",
 30: 'asking',
 31: 'me',
 32: 'out.',
 33: 'so',
 34: 'cute.',
 35: 'what',
 36: 'your',
 37: 'name',
 38: 'again?',
 39: 'no,',
 40: "it's",
 41: 'my',
 42: 'fault',
 43: '--',
 44: "didn't",
 45: 'have',
 46: 'a',
 47: 'proper',
 48: '---',
 49: 'thing',
 50: 'is,',
 51: "i'm",
 52: 'at',
 53: 'mercy',
 54: 'of',
 55: 'particularly',
 56: 'breed',
 57: 'loser.',
 58: 'sister.',
 59: 'cannot',
 60: 'date',
 61: 'until',
 62: 'she',
 63: 'does.',
 64: 'why?',
 65: 'mystery.',
 66: 'used',
 67: 'to',
 68: 'be',
 69: 'really',
 70: 'popular',
 71: 'when',
 72: 'started',
 73: 'high',
 74: 'school,',
 75: 'th

In [117]:
questions_2_int = []
for question in clean_questions:
    ints = []
    for word in question.split():
        if word not in questionword2int:
            ints.append(questionword2int['<OUT>'])
        else:
            ints.append(questionword2int[word])
    questions_2_int.append(ints)

answers_2_int = []
for answer in clean_answers:
    ints = []
    for word in answer.split():
        if word not in answerword2int:
            ints.append(answerword2int['<OUT>'])
        else:
            ints.append(answerword2int[word])
    answers_2_int.append(ints)

In [114]:
#answers_2_int

Sort by the length of the questions

In [119]:
sorted_clean_questions = []
sorted_clean_answers = []
for length in range(1,25 + 1):
    for i in enumerate(questions_2_int):#first is index,second itself
        if len(i[1]) == length:
            sorted_clean_questions.append(questions_2_int[i[0]])
            sorted_clean_answers.append(answers_2_int[i[0]])

In [122]:
#sorted_clean_answers

In [123]:
#sorted_clean_questions

In [124]:
#

EncoderRNN